In [1]:
import os
import random
from datetime import datetime
from collections import Counter

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa

from utils import call_backs, misc, preprocessing
from custom_layers.arcface_loss import ArcMarginProduct

### Config

In [2]:
class config:
    # GENERAL
    RANDOM_SEED = 5
    TENSOR_LOG_DIR = 'logs'
    SAVE_DIR = 'saved_models'

    # DATA
    INPUT_SIZE = (28,28,1)
    NUM_CLASSES = 10

    # MODEL
    OUTPUT_EMB = 64
    MIDDLE_EMB = 256

    # TRAINING
    BATCH_SIZE = 32
    LR = .000005

misc.seed_everything(config.RANDOM_SEED)

### Load dataset

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

x_train = tf.expand_dims(tf.convert_to_tensor(x_train),-1)
y_train = tf.expand_dims(tf.convert_to_tensor(y_train),-1)

In [4]:
y_train.shape

TensorShape([60000, 1])

In [5]:
class_count = Counter(np.array(tf.reshape(y_train, [60000,])))
print(class_count)

Counter({9: 6000, 0: 6000, 3: 6000, 2: 6000, 7: 6000, 5: 6000, 1: 6000, 6: 6000, 4: 6000, 8: 6000})


In [6]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(preprocessing.normalize).map(preprocessing.arcface_format).batch(config.BATCH_SIZE)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).map(preprocessing.normalize).map(preprocessing.arcface_format).batch(config.BATCH_SIZE)

### Define model 

In [7]:
from custom_layers.arcface_loss import ArcMarginProduct
# allows 2 inputs and 2 outputs

def get_debug_model(s = 10, m = .25):
 #------------------
    # Definition of placeholders
    inp = tf.keras.layers.Input(shape = config.INPUT_SIZE, name = 'inp1')
    label = tf.keras.layers.Input(shape = (), name = 'inp2')

    # Definition of layers
    
    #TODO: reasearch filters, get better understanding
    layer_conv1 = tf.keras.layers.Conv2D(filters = 24, kernel_size = (2,2), input_shape = config.INPUT_SIZE, activation ='relu')
    layer_pool1 = tf.keras.layers.MaxPool2D((2,2))
    layer_conv2 = tf.keras.layers.Conv2D(filters = 12, kernel_size = (2,2), activation ='relu')
    layer_pool2 = tf.keras.layers.MaxPool2D((2,2))
    # layer_conv3 = tf.keras.layers.Conv2D(filters = 8, kernel_size = (2,2), activation ='relu')
    # layer_pool3 = tf.keras.layers.MaxPool2D((2,2))
    layer_flatten = tf.keras.layers.Flatten()
    layer_dense1 = tf.keras.layers.Dense(config.MIDDLE_EMB)
    # layer_dense2 = tf.keras.layers.Dense(config.NUM_CLASSES)
    layer_arcface = ArcMarginProduct(n_classes=config.NUM_CLASSES, s=s, m=m)
    layer_softmax = tf.keras.layers.Softmax(dtype='float16', name='head_output')

    if config.MIDDLE_EMB != config.OUTPUT_EMB:
        layer_adaptive_pooling = tfa.layers.AdaptiveAveragePooling1D(config.OUTPUT_EMB)
    else:
        layer_adaptive_pooling = tf.keras.layers.Lambda(lambda x: x)  # layer with no operation

    #------------------
    # Definition of entire model      --- w/o arcface accuracy is ~.87, try to get at least close to that
    backbone_output = layer_conv1(inp)
    backbone_output = layer_pool1(backbone_output)
    backbone_output = layer_conv2(backbone_output)
    backbone_output = layer_pool2(backbone_output)
    embed = layer_flatten(backbone_output)
    embed = layer_dense1(embed)
    
    # Training head
    # head_output = layer_dense2(embed)
    head_output = layer_arcface(embed)
    head_output = layer_softmax(head_output)
    
    # Inference
    emb_output = layer_adaptive_pooling(embed)

    model = tf.keras.models.Model(inputs = [(inp, label)], outputs = [head_output, emb_output]) # whole architecture

    return model

In [8]:
debug_model = get_debug_model(s=32, m = .15)
debug_model.summary()

OperatorNotAllowedInGraphError: Exception encountered when calling layer "arc_margin_product" (type ArcMarginProduct).

in user code:

    File "c:\Users\Jack\Desktop\GUIE\guie_challenge\guie_challenge\custom_layers\arcface_loss.py", line 51, in call  *
        X, y = inputs #

    OperatorNotAllowedInGraphError: Iterating over a symbolic `tf.Tensor` is not allowed in Graph execution. Use Eager execution or decorate this function with @tf.function.


Call arguments received by layer "arc_margin_product" (type ArcMarginProduct):
  • inputs=tf.Tensor(shape=(None, 256), dtype=float32)

In [ ]:
debug_model.compile(
        optimizer = tf.keras.optimizers.Adam(learning_rate = config.LR),
        loss = {'head_output':tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)},
        metrics = {'head_output':[tf.keras.metrics.SparseCategoricalAccuracy(),tf.keras.metrics.SparseTopKCategoricalAccuracy(k=3)]},
        )

steps_per_epoch = len(train_dataset) // config.BATCH_SIZE  // 20     # "//20" means that the lr is update every 0.1 epoch.
validation_steps = len(test_dataset) // config.BATCH_SIZE
if len(test_dataset) % config.BATCH_SIZE != 0:
    validation_steps += 1
print(steps_per_epoch, validation_steps)

### Callbacks

In [ ]:
# tensorboard

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1,
                        #  write_graph=True,
                        #  write_images=True,
                        update_freq='epoch',
                        #  profile_batch=2,
                        #  embeddings_freq=1
                        )

#emb_callback = call_backs.EmbeddingCallback(x_test, y_test, save_dir = log_dir+'/emb/', embedding_dim = config.OUTPUT_EMB)

### Training

In [ ]:
history = debug_model.fit(
        train_dataset,
        epochs=35,
        validation_steps = validation_steps,
        validation_data = test_dataset,
        verbose=1,
        #callbacks=[tensorboard_callback]
    )

In [ ]:
print("Test examples:",len(x_test))
pred_class, pred_emb = debug_model.predict(x_test)

# argmax returns largest element's index
pred_class = tf.argmax(pred_class, axis=1)
true_class = y_test


In [ ]:
confusion_matrix = {}
embedding_data = []

for i in range(config.NUM_CLASSES):
    confusion_matrix[i] = {}
    for ii in range(config.NUM_CLASSES):
        confusion_matrix[i][ii] = 0

from annoy import AnnoyIndex
tree = AnnoyIndex(config.OUTPUT_EMB, 'euclidean')

for i,pred_tensor, y_array, embedding in zip(range(len(true_class)),pred_class,true_class,pred_emb):
    # come out in annoying type
    pred = int(pred_tensor)
    y = int(y_array)

    tree.add_item(i, embedding)

    assert y < config.NUM_CLASSES and pred < config.NUM_CLASSES
    embedding_data.append({'annoy_idx':i,'true_class':y,'pred_class':pred,'embedding':embedding})
    confusion_matrix[y][pred]+=1

In [ ]:
# import matplotlib.pyplot as plt
# cm_df = pd.DataFrame(confusion_matrix)

# fig, ax = plt.subplots(figsize=(4,6))
# ax.matshow(cm_df, cmap=plt.cm.Reds, alpha=0.7)
# for i in range(cm_df.shape[0]):
#     for j in range(cm_df.shape[1]):
#         ax.text(x=j, y=i,s=cm_df[i][j], va='center', ha='center', size='large')
 
# plt.xlabel('Predictions', fontsize=18)
# plt.ylabel('Actuals', fontsize=18)
# plt.title('Confusion Matrix', fontsize=18)
# plt.show()

In [ ]:
emb_df = pd.DataFrame(embedding_data)
emb_df.head()

In [ ]:
def dist_to_origin(embedding):
    return np.sqrt(np.dot(embedding,embedding))

emb_df['length']=emb_df['embedding'].apply(dist_to_origin)
emb_df['normed_embeddings']=emb_df['embedding']/emb_df['length']

In [ ]:
emb_df.head()

In [ ]:
tree.build(20)

In [ ]:
def n_neighbors(annoy_idx, neighbor_count=5):
    # the closest embedding is always the same exact embedding, must exclu
    neighbor_count+=1
    return tree.get_nns_by_item(annoy_idx, neighbor_count)[1:]

def neighbor_classes(row, emb_df):
    return emb_df.loc[emb_df['annoy_idx'].isin(row.nearest_neighbors),'true_class'].to_list()
    
def matching_neighbors(row):
    return len([True for neighbor_class in row.neighbor_classes if row.true_class == neighbor_class])

emb_df['nearest_neighbors'] = emb_df['annoy_idx'].apply(n_neighbors)
emb_df['neighbor_classes'] = emb_df.apply(lambda row: neighbor_classes(row,emb_df), axis=1)
emb_df['matching_neighbors'] = emb_df.apply(lambda row: matching_neighbors(row), axis=1)

In [ ]:
def competition_score(emb_df, neighbor_count):
    # Average of (correct neighbors / looked at neighbors)
    # ex: 1/N * (  sum(  1/neighbor_count * per_item_match_count  )  ) = 1/N * ( 1/neighbor_count * total_match_count)
    total_matches = emb_df['matching_neighbors'].sum()
    score = total_matches / neighbor_count / len(emb_df)
    return score

In [ ]:
competition_score(emb_df,5)

In [ ]:
emb_df['correct_prediction'] = emb_df['true_class']==emb_df['pred_class']

In [ ]:
print(emb_df['correct_prediction'].value_counts())